In [65]:
import json 
import numpy as np 
import pandas as pd 
import tensorflow as tf
import string
import nltk
from nltk import *
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D , Activation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder


In [46]:
with open("Intent.json") as train_file:
    df= json.load(train_file)



# TOKENIZATION

In [47]:
df["intents"][0]["intent"]

'Greeting'

In [54]:
tags = []
inputs = []
responses = {}

for intent in df['intents']:
    responses[intent['intent']] = intent['responses']
    for lines in intent['text']:
        inputs.append(lines)
        tags.append(intent['intent'])
df = pd.DataFrame({'inputs': inputs, 'tags': tags})
df

,inputs,tags
0,Hi,Greeting
1,Hi there,Greeting
2,Hola,Greeting
3,Hello,Greeting
4,Hello there,Greeting
...,...,...
138,Can you prove you have a conscious,SelfAware
139,Can you prove you are self-aware please,SelfAware
140,Can you prove you are self aware please,SelfAware
141,Can you prove you have a conscious please,SelfAware


In [57]:
df["inputs"] = df["inputs"].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])

In [58]:
df["inputs"] = df["inputs"].apply(lambda wrd:"".join(wrd))

In [59]:
tokenizer = Tokenizer(num_words=2000)

In [74]:
index_list = df.index.tolist()

In [72]:
nltk.word_tokenize(index_list)

TypeError: expected string or bytes-like object

In [32]:
nltk.word_tokenize("how are you doing ? ")
#this will tokenize the phrase "how are you doing ? "

['how', 'are', 'you', 'doing', '?']

# DELETE OR ADD STOPWORDS

In [60]:
from nltk.corpus import stopwords

In [61]:
stop_words = stopwords.words("english")

In [62]:
print(stop_words)
#this will show all the default stop word in the language english

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [63]:
stop_words.append("work")
stop_words
#this will add or remove a stop word

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

# STEMMING

In [39]:
from nltk.stem import PorterStemmer

In [40]:
stemmers = PorterStemmer()

In [42]:
input_str = "i don't know what i am doing please help"

In [43]:
input_str = nltk.word_tokenize(input_str)

In [45]:
for word in input_str :
    print(stemmers.stem(word))

i
do
n't
know
what
i
am
do
pleas
help


# LEMMATIZATION

In [47]:
from nltk.stem import WordNetLemmatizer

In [48]:
lemmatizer = WordNetLemmatizer()

In [49]:
input_str = "so is it the same than stemming"

In [50]:
input_str = nltk.word_tokenize(input_str)

In [52]:
for word in input_str :
    print(lemmatizer.lemmatize(word))

so
is
it
the
same
than
stemming


In [53]:
lemmatizer = WordNetLemmatizer()

In [54]:
for index,row in data.iterrows():
    filter_sentence = []
    sentence = row["Title"]

AttributeError: 'dict' object has no attribute 'iterrows'

In [11]:
#Define Sequential Model
model = Sequential() 
#Create input layer
model.add(Dense(32, input_dim=784))
#Create hidden layer 
model.add(Activation('relu')) 
#Create Output layer
model.add(Activation('sigmoid')) 

In [12]:
#Compiling the model with a mean squared error loss and RMSProp #optimizer
model.compile(optimizer='rmsprop',loss='mse')

In [55]:
model.fit(input_str, labels, epochs=10, batch_size=32)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'str'>"}), (<class 'list'> containing values of types set())

In [6]:
label_enc = LabelEncoder()

In [7]:
label_enc.fit()

ValueError: y should be a 1d array, got an array of shape (3725, 2) instead.